# Using LM-inspector with a WSD-classifier

Import libraries

In [1]:
import torch
from transformers import AutoConfig, AutoModel, AutoTokenizer

from lm_inspect import LanguageModelInspector
from word_sense_disambiguation import bert_encoder, label_encoder, Xval as Xtest, Yval as Ytest

/home/felix/Workspaces/PhD/lm-inspector/.vn/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1938: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Load trained classifier from binary file.

In [2]:
seq = torch.nn.Sequential(
            bert_encoder,
            torch.nn.Dropout(0.2),
            torch.nn.Linear(bert_encoder.output_size, out_features=358)
        ).to('cpu')
state_dict = torch.load('models/KB-bert-swedish-cased-wsd.pt', map_location=torch.device('cpu'))
seq.load_state_dict(state_dict)

<All keys matched successfully>

Load transformers config and tokenizer

In [3]:
config = AutoConfig.from_pretrained('KB/bert-base-swedish-cased',
                                    output_hidden_states=True,
                                    output_attentions=True
                                    )
tokenizer = AutoTokenizer.from_pretrained('KB/bert-base-swedish-cased', config=config)

Get the positions of the ambigious words

In [4]:
input_ids = [x['pos'] for x in Xtest]

Initialize LM-inspector object and set the configuration.

In [ ]:
inspector = LanguageModelInspector(seq, Xtest, Ytest, tokenizer, label_encoder, device = 'cpu')
inspector.configure(label='följa_1_3_a', layer=[0, 6, 11], head=[0,6], input_id=input_ids)

Evaluating data
1520 / 10000
Done.


Apply method on the current configuration

In [ ]:
inspector.topk_most_attended_to(k=5, return_type="all")

Visualize results scope-wise.

In [ ]:
inspector.topk_most_attended_to(k=5, return_type="scope", visualize=True)